In [ ]:
import torch
import numpy as np
from brainscore_vision import load_model, load_stimulus_set
model = load_model('alexnet')

: 

In [ ]:
model.__dict__

In [3]:
act_model = model.activations_model
act_model.to('cuda')

In [ ]:
act_model.__dict__

In [ ]:
for i in act_model.layers():
    print(i)

In [6]:
# just in case this is how to access the baseline model
model_ = act_model._model

In [7]:
conv_list = ['features.0', 'features.3', 'features.6', 'features.8', 'features.10']

In [ ]:
act_model._model.features[0]

In [9]:
stimuli = load_stimulus_set('FreemanZiemba2013.aperture-public')  # load some images for the model to look at

In [10]:
transforms = model.activations_model._extractor.preprocess

In [11]:
model_extraction = act_model(stimuli, conv_list)

In [ ]:
model_extraction

In [13]:
def generate_conv_layer_configs(xarray_data):
    conv_layer_configs = {}
    
    # Extract unique layers
    unique_layers = np.unique(xarray_data['layer'].values)
    
    for layer in unique_layers:
        # Select data for the specific layer
        layer_data = xarray_data.sel(neuroid=xarray_data['layer'] == layer)
        
        # Compute channels, height, and width
        channels = len(np.unique(layer_data['channel'].values))
        height = len(np.unique(layer_data['channel_y'].values))
        width = len(np.unique(layer_data['channel_x'].values))
        
        # Add to config dictionary
        conv_layer_configs[layer] = {
            'channels': channels,
            'height': height,
            'width': width
        }
    
    return conv_layer_configs

In [14]:
conv_layer_configs = generate_conv_layer_configs(model_extraction)

In [ ]:
print(conv_layer_configs)

In [16]:
import numpy as np

def reshape_xarray_to_4d(xarray_data, conv_layer_configs):
    reshaped_data = {}

    # Extract metadata
    presentation_size = len(xarray_data.presentation)

    for layer, config in conv_layer_configs.items():
        # Filter neuroids corresponding to the current layer
        layer_data = xarray_data.sel(neuroid=xarray_data['layer'] == layer)
        
        # Validate dimensions
        expected_neuroids = config['channels'] * config['height'] * config['width']
        if len(layer_data.neuroid) != expected_neuroids:
            raise ValueError(
                f"Layer {layer} expects {expected_neuroids} neuroids, but got {len(layer_data.neuroid)}."
            )

        # Reshape to (batch_size, channels, height, width)
        reshaped_layer = layer_data.values.reshape(
            presentation_size, config['channels'], config['height'], config['width']
        )
        reshaped_data[layer] = reshaped_layer

    return reshaped_data


In [17]:
reshaped_4d = reshape_xarray_to_4d(model_extraction, conv_layer_configs)

In [ ]:
reshaped_4d

In [19]:
# How to make flatten version (2D)

# Get the list of layers
layers = model_extraction['layer'].values

# Extract features by layer
features_by_layer = {layer: model_extraction.sel(neuroid=model_extraction['layer'] == layer).data for layer in conv_list}

In [20]:
# Create a dictionary to save in .npz format
npz_data = {layer_name: features for layer_name, features in reshaped_4d.items()}

# Save as an npz file
np.savez('{}.npz'.format(model.identifier), **npz_data)

In [21]:
# Load the npz file
data = np.load('alexnet.npz')

# Access data of a specific layer
features_0 = data['features.0']

In [ ]:
features_0

In [23]:
import numpy as np
#from sklearn.decomposition import PCA

class EigenSpectraCalculator:
    def __init__(self, layer_features):
        """
        layer_features: Dictionary containing {layer_name: feature_list}
        feature_list: A 2D numpy array of shape (num_samples, feature_dim)
        """
        self.layer_features = layer_features
        self._layer_eigenspectra = {}

    def flatten(self, layer_output):
      flattened = layer_output.reshape(layer_output.shape[0], -1)

      return flattened

    def global_avg_pooling(self, layer_output):
      if len(layer_output.shape) != 4:
        raise ValueError("Input features must be a 4-dim with shape (B, C, H, W).")
    
      # Compute the global average pooling over the last two dimensions (H, W)
      pooled_features = layer_output.mean(axis=(2, 3))
      print(pooled_features.shape)
      
      return pooled_features

    def compute_eigenspectra(self, use_global_avg_pooling=False):
      """
      Compute eigenspectra for each layer based on the feature data using SVD.
      """
      self._layer_eigenspectra = {}
      for layer, features in self.layer_features.items():
          # global average pooling
          if use_global_avg_pooling:
            features = self.global_avg_pooling(features)
          else:
            features = self.flatten(features)

          print(features.shape)

          # pca = PCA()
          # pca.fit(features)

          # eigenspectrum = pca.explained_variance_

          # Center the features
          centered_features = features - np.mean(features, axis=0)

          # Compute the SVD
          S = np.linalg.svd(centered_features, compute_uv=False)

          eigenspectrum = (S ** 2) / (centered_features.shape[0] - 1)

          #Store the eigenvalues as eigenspectra for the layer
          self._layer_eigenspectra[layer] = eigenspectrum

    def effective_dimensionalities(self):
        """
        Calculate effective dimensionalities for each layer.
        Effective Dimensionality = (Sum of eigenvalues)^2 / Sum of squared eigenvalues
        """
        effdims = {
            layer: eigspec.sum() ** 2 / (eigspec**2).sum()
            for layer, eigspec in self._layer_eigenspectra.items()
        }
        return effdims

In [ ]:
calculator_avg = EigenSpectraCalculator(reshaped_4d)
calculator_avg.compute_eigenspectra(use_global_avg_pooling=True)

In [ ]:
calculator_avg.effective_dimensionalities()

In [ ]:
calculator = EigenSpectraCalculator(reshaped_4d)
calculator.compute_eigenspectra()

In [ ]:
calculator.effective_dimensionalities()

In [ ]:
resnext = load_model('resnext101_32x48d_wsl')

In [ ]:
convnet1 = load_model('convnext_tiny_imagenet_full_seed-0')

In [ ]:
convnet1.activations_model

In [ ]:
for layer in convnet1.layers:
    print(layer)

In [ ]:
resnet = load_model('resnet152_imagenet_full')